In [13]:
# Gerekli kütüphaneleri içe aktar
import pandas as pd # Veri manipülasyonu için Pandas kütüphanesi
import torch # Derin öğrenme operasyonları için PyTorch kütüphanesi
from transformers import AutoTokenizer, AutoModelForSequenceClassification # Hugging Face Transformers kütüphanesinden tokenizer ve sınıflandırma modeli sınıfları

# 1. Adım: Model Adını Seç
# Kullanılacak önceden eğitilmiş BERT modelinin adı belirlenir.
# "dbmdz/bert-base-turkish-cased" Türkçe için optimize edilmiş bir modeldir.
model_name = "dbmdz/bert-base-turkish-cased" 

# 2. Adım: Tokenizer'ı Yükle
print(f"'{model_name}' için tokenizer yükleniyor...")
# Belirlenen model adına uygun tokenizer yüklenir.
# Tokenizer, metinleri modelin anlayabileceği sayısal formata (token ID'lerine) dönüştürür.
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Tokenizer başarıyla yüklendi.")

# 3. Adım: Modeli Yükle
print(f"'{model_name}' modeli yükleniyor ve sınıflandırma başlığı ekleniyor...")
# Belirlenen model adıyla önceden eğitilmiş model yüklenir.
# `num_labels=2` parametresi, modelin ikili sınıflandırma (doğru/yanlış) yapacağını belirtir.
# Bu, modelin üzerine yeni bir sınıflandırma katmanı eklenmesini sağlar.
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
print("Model başarıyla yüklendi.")

# Modeli GPU'ya taşıma kontrolü ve taşıma
if torch.cuda.is_available():
    print("GPU (CUDA) algılandı. Model GPU'ya taşınıyor...")
    model.to('cuda') # Modeli GPU'ya taşı
    print(f"Model şu cihazda: {model.device}")
    # Kullanılan GPU'nun adını da yazdırabiliriz
    print(f"Kullanılan GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU (CUDA) algılanmadı. Model CPU'da kalacak.")

# 4. Adım: Yüklemeyi Kontrol Et (İsteğe bağlı)
print("\n--- Yükleme Kontrolü ---")
print(f"Yüklenen Model Adı: {model_name}")
print(f"Yüklenen Tokenizer Tipi: {type(tokenizer)}")
print(f"Yüklenen Model Tipi: {type(model)}")
print(f"Modelin şu anki cihazı: {model.device}") # Modelin hangi cihazda olduğunu kontrol et

# Örnek bir metni kullanarak tokenizer'ın ve modelin çalıştığını göster
sample_text = "Yapay zeka modelleri, metinleri analiz etmekte çok başarılıdır."
# Örnek metin tokenizer ile işlenir.
# `return_tensors="pt"` çıktının PyTorch tensor'u olmasını sağlar.
# `truncation=True` uzun metinleri modelin maksimum giriş uzunluğuna göre keser.
# `padding=True` kısa metinleri aynı uzunluğa kadar doldurur.
inputs = tokenizer(sample_text, return_tensors="pt", truncation=True, padding=True)

print(f"\nÖrnek Metin: '{sample_text}'")
# Tokenizer tarafından metnin ayrıştırıldığı token'ları (kelime parçalarını) gösterir.
print(f"Token'lar: {tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])}")
# Token'ların sayısal ID'lerini gösterir.
print(f"Input IDs: {inputs['input_ids']}")

# Eğer model GPU'daysa, input'ları da GPU'ya taşı
if model.device.type == 'cuda':
    inputs = {k: v.to('cuda') for k, v in inputs.items()}
    print(f"Giriş tensorleri şu cihaza taşındı: {inputs['input_ids'].device}")

# Modelden tahmin al (model henüz veri setinle eğitilmediği için anlamsız bir tahmin dönecektir)
# `torch.no_grad()` bağlam yöneticisi, tahmin yaparken gradyan hesaplamasını kapatır.
# Bu, bellek ve hesaplama açısından verimlilik sağlar çünkü eğitim yapılmamaktadır.
with torch.no_grad(): 
    outputs = model(**inputs) # Modelden çıktı alınır

logits = outputs.logits # Modelin ham çıktıları (logitler)
# Logitler softmax fonksiyonundan geçirilerek olasılıklara dönüştürülür.
# `dim=1` olasılıkların sınıf boyutunda hesaplanmasını sağlar.
probabilities = torch.softmax(logits, dim=1) 

print(f"\nModel Çıkışı (Logitler): {logits}")
print(f"Model Çıkışı (Olasılıklar): {probabilities}")
# En yüksek olasılığa sahip sınıfın indeksini bulur (0 veya 1).
print(f"Tahmini Sınıf İndeksi (0 veya 1): {torch.argmax(probabilities, dim=1).item()}")

'dbmdz/bert-base-turkish-cased' için tokenizer yükleniyor...
Tokenizer başarıyla yüklendi.
'dbmdz/bert-base-turkish-cased' modeli yükleniyor ve sınıflandırma başlığı ekleniyor...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model başarıyla yüklendi.
GPU (CUDA) algılandı. Model GPU'ya taşınıyor...
Model şu cihazda: cuda:0
Kullanılan GPU: NVIDIA GeForce RTX 3050 6GB Laptop GPU

--- Yükleme Kontrolü ---
Yüklenen Model Adı: dbmdz/bert-base-turkish-cased
Yüklenen Tokenizer Tipi: <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>
Yüklenen Model Tipi: <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'>
Modelin şu anki cihazı: cuda:0

Örnek Metin: 'Yapay zeka modelleri, metinleri analiz etmekte çok başarılıdır.'
Token'lar: ['[CLS]', 'Yapay', 'zeka', 'modelleri', ',', 'metin', '##leri', 'analiz', 'etmekte', 'çok', 'başarılı', '##dır', '.', '[SEP]']
Input IDs: tensor([[    2, 27980, 14348,  6690,    16,  8111,  2065,  6516, 11532,  2140,
          4165,  2077,    18,     3]])
Giriş tensorleri şu cihaza taşındı: cuda:0

Model Çıkışı (Logitler): tensor([[-0.3814,  0.4137]], device='cuda:0')
Model Çıkışı (Olasılıklar): tensor([[0.3111, 0.6889]], device='cuda:0')
Tahm

In [14]:
import pandas as pd # Veri manipülasyonu ve analizi için Pandas kütüphanesi
from sklearn.model_selection import train_test_split # Veri setini eğitim, test vb. parçalara ayırmak için
from datasets import Dataset # Hugging Face'in optimizasyonlu veri seti sınıfı
import re # Metin temizleme (regular expressions) işlemleri için

# 1. Adım: Veri setini yükle
# İndirilen 'fake_news.csv' dosyasının yolu belirtilir ve Pandas ile DataFrame olarak okunur.
file_path = 'fake_news.csv'

try:
    # Dosya okuma sırasında kodlama hatası olmaması için deneme-yanılma yapalım
    try:
        df = pd.read_csv(file_path, encoding='utf-8') # Önce yaygın olan utf-8 kodlamasını deneyelim
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='latin1') # utf-8 olmazsa latin1 kodlamasını deneyelim
        print("Dosya UTF-8 yerine 'latin1' kodlamasıyla yüklendi.")
    except Exception as e: # Diğer olası hataları yakala
        print(f"Dosya okunurken beklenmeyen bir hata oluştu: {e}")
        raise # Hatayı yeniden fırlat ki dış except blokları yakalayabilsin

    print("Veri Seti Başarıyla Yüklendi!\n")

    # İLK KONTROL: Veri setinin ilk birkaç satırını ve sütun adlarını göstererek yapısını kontrol et
    print("Veri setinin ilk 5 satırı:\n", df.head())
    print("\nVeri setinin sütunları:", df.columns.tolist()) # .tolist() ekledim daha okunaklı olsun
    print("-" * 50)

    # 'fake_news.csv' dosyası için beklenen sütun adları: 'news' ve 'type'
    text_column = 'news'
    label_column = 'type'

    # Sütunların gerçekten var olup olmadığını kontrol et
    if text_column not in df.columns:
        raise ValueError(f"'{text_column}' sütunu veri setinde bulunamadı. Lütfen dosya yapısını kontrol edin.")
    if label_column not in df.columns:
        raise ValueError(f"'{label_column}' sütunu veri setinde bulunamadı. Lütfen dosya yapısını kontrol edin.")

    print(f"Metin sütunu olarak '{text_column}' kullanılıyor.")
    print(f"Etiket sütunu olarak '{label_column}' kullanılıyor.")
    print("-" * 50)


    # 2. Adım: Metinleri temizle
    # Metin ön işleme için bir fonksiyon tanımlanır.
    def clean_text(text):
        if not isinstance(text, str):
            return "" # Metin olmayan değerleri (örn. NaN) boş stringe çevirerek hata oluşmasını engeller.
        text = text.lower() # Tüm metni küçük harfe dönüştürerek büyük/küçük harf farkını ortadan kaldırır.
        text = re.sub(r'http\S+|www\S+|t\.co/\S+', '', text) # URL'leri metinden kaldırır.
        text = re.sub(r'[^\w\s]', '', text) # Alfabetik ve boşluk karakteri dışındaki tüm noktalama işaretlerini kaldırır.
        text = re.sub(r'\s+', ' ', text).strip() # Birden fazla boşluğu tek boşluğa indirir ve baştaki/sondaki boşlukları kaldırır.
        return text

    # 'news' sütunundaki her bir metne clean_text fonksiyonunu uygula ve yeni 'temiz_metin' sütunu oluştur.
    df['temiz_metin'] = df[text_column].apply(clean_text)

    # 3. Adım: Etiketleri sayısal hale dönüştür
    # 'real' ve 'fake' string etiketlerini modelin anlayabileceği sayısal değerlere dönüştürmek için bir sözlük tanımlanır.
    # 'real' (doğru) için 0, 'fake' (yanlış/sahte) için 1 atanır.
    label_mapping = {'real': 0, 'fake': 1}

    # Sadece 'real' veya 'fake' olan satırları filtrele
    # Bu, 'type' sütununda başlık satırı gibi istenmeyen değerler varsa onları temizler.
    df = df[df[label_column].isin(['real', 'fake'])].copy()

    # 'type' sütunundaki etiketleri 'label_mapping'e göre sayısal 'etiket_id' sütununa dönüştür.
    df['etiket_id'] = df[label_column].map(label_mapping)

    # NaN (Not a Number) etiketleri varsa kaldır (veri setinde eksik veya eşleşmeyen etiketler olabilir).
    # Bu adım, temizlenmiş etiket sütununda eksik değer bulunmamasını sağlar.
    df.dropna(subset=['etiket_id'], inplace=True)
    df['etiket_id'] = df['etiket_id'].astype(int) # Etiket ID'lerini tam sayı (integer) tipine çevir.

    # Sadece modelin kullanacağı 'temiz_metin' ve 'etiket_id' sütunlarını seç.
    # Hugging Face Transformer modelinin beklentisine uygun olarak sütun adlarını 'text' ve 'labels' olarak yeniden adlandır.
    df = df[['temiz_metin', 'etiket_id']].rename(columns={'temiz_metin': 'text', 'etiket_id': 'labels'})

    # DİAGNOSTİK KONTROL: train_test_split öncesi DataFrame'in boş olup olmadığını kontrol et
    if len(df) == 0:
        raise ValueError(
            "UYARI: Veri temizleme ve etiketleme adımlarından sonra DataFrame boş kaldı. "
            "Bu, dosyanızdaki etiketlerin 'real' veya 'fake' dışında değerler içerdiği veya "
            "metin/etiket sütunlarının boş olduğu anlamına gelebilir."
            "\nŞu anki DataFrame boyutu: 0"
        )
    else:
        print(f"\nVeri temizleme sonrası DataFrame boyutu: {len(df)} satır.")
        print("DataFrame'in ilk 3 satırı (temizlenmiş):\n", df.head(3))
        print("Etiket dağılımı (temizlenmiş):\n", df['labels'].value_counts())
        print("-" * 50)


    # 4. Adım: Veri Setini Eğitim, Doğrulama (Validation) ve Test Setlerine Ayır
    # Veri setini %80 eğitim, %10 doğrulama ve %10 test oranlarında ayır.
    # `random_state=42` sonuçların tekrarlanabilir olmasını sağlar.
    # `stratify=df['labels']` etiket dağılımının her alt kümede aynı kalmasını sağlar, bu özellikle dengesiz veri setlerinde önemlidir.
    train_val_df, test_df = train_test_split(df, test_size=0.10, random_state=42, stratify=df['labels'])
    # Eğitim ve doğrulama setini kendi içinde tekrar bölerek %10'luk doğrulama setini oluştururuz.
    train_df, val_df = train_test_split(train_val_df, test_size=0.10, random_state=42, stratify=train_val_df['labels'])

    # Ayrılan setlerin boyutlarını (satır sayılarını) kontrol et
    print("Eğitim seti boyutu:", len(train_df))
    print("Doğrulama (Validation) seti boyutu:", len(val_df))
    print("Test seti boyutu:", len(test_df))

    # Ayrılan setlerdeki etiket dağılımını kontrol et (stratify'ın işe yaradığını doğrulamak için)
    print("\nEğitim seti etiket dağılımı:\n", train_df['labels'].value_counts())
    print("Doğrulama seti etiket dağılımı:\n", val_df['labels'].value_counts())
    print("Test seti etiket dağılımı:\n", test_df['labels'].value_counts())

    # Pandas DataFrame'lerini Hugging Face Dataset objelerine dönüştür.
    train_dataset = Dataset.from_pandas(train_df, preserve_index=False)
    val_dataset = Dataset.from_pandas(val_df, preserve_index=False)
    test_dataset = Dataset.from_pandas(test_df, preserve_index=False)

    # Dönüştürülen Dataset objelerinin ilk bilgilerini göster
    print("\nHugging Face Dataset formatında eğitim seti:\n", train_dataset)
    print("Hugging Face Dataset formatında doğrulama seti:\n", val_dataset)
    print("Hugging Face Dataset formatında test seti:\n", test_dataset)

except FileNotFoundError:
    print(f"Hata: '{file_path}' dosyası bulunamadı. Lütfen dosya adını ve yolunu kontrol edin.")
    print("Dosya adının 'fake_news.csv' olduğundan ve aynı dizinde bulunduğundan emin olun.")
except pd.errors.EmptyDataError:
    print(f"Hata: '{file_path}' dosyası boş.")
except ValueError as ve:
    print(f"Veri setinin yapısında hata: {ve}")
except Exception as e:
    print(f"Beklenmeyen bir hata oluştu: {e}")

Veri Seti Başarıyla Yüklendi!

Veri setinin ilk 5 satırı:
    type                                               news  \
0  real  Gazze’de sağlanan ateşkes sonrası Suriye'nin b...   
1  real  Hamas ve İsrail arasında anlaşma sağlandığının...   
2  real  Gazze'de Hamas ile İsrail arasında varılan ate...   
3  real  Hamas ve İsrail arasında anlaşma sağlandığının...   
4  real  İsrail’in Gazze’ye düzenlediği saldırılarda ya...   

          shared_site_name source  
0  https://t.co/GVwaGyT1lG     AA  
1  https://t.co/omyAR4NXll     AA  
2  https://t.co/UkQJnOnWSW     AA  
3  https://t.co/afEqmk38Pp     AA  
4  https://t.co/xjSuhkjOp5     AA  

Veri setinin sütunları: ['type', 'news', 'shared_site_name', 'source']
--------------------------------------------------
Metin sütunu olarak 'news' kullanılıyor.
Etiket sütunu olarak 'type' kullanılıyor.
--------------------------------------------------

Veri temizleme sonrası DataFrame boyutu: 693 satır.
DataFrame'in ilk 3 satırı (temizlenmiş):
 

In [15]:
from transformers import AutoTokenizer # Sadece AutoTokenizer sınıfını içe aktarıyoruz (diğerleri zaten yukarıda içe aktarılmış olmalı)

# Tokenizer objesi, daha önceki hücrelerde 'model_name' değişkeni kullanılarak yüklendi.
# Metinleri token'lara dönüştürmek için bir fonksiyon tanımlarız.
# Bu fonksiyon, Hugging Face Dataset'in 'map' metodu ile kullanılacaktır.
def tokenize_function(examples):
    # 'examples["text"]' içinde bulunan metinleri tokenizer ile işler.
    # truncation=True: Metin, modelin maksimum giriş uzunluğunu (örneğin BERT için 512) aşarsa kesilir.
    # padding="max_length": Tüm metinleri aynı uzunluğa (max_length) kadar doldurur.
    # max_length=256: Tüm metinlerin maksimum uzunluğunu 256 token olarak ayarlar.
    # Bu değer, modelin kaldırabileceği ve projenin ihtiyaçlarına uygun bir dengedir.
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

# Metinleri Token'lara Ayırma 
print("Veri setleri token'lara ayrılıyor...")
# 'map' metodu, veri setindeki her örneğe tokenize_function'ı uygular.
# 'batched=True' parametresi, fonksiyonun aynı anda birden fazla örnek üzerinde çalışmasını sağlayarak hızı artırır.
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True) # Doğrulama seti de token'lara ayrılır
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True) # Test seti de token'lara ayrılır
print("Tokenizasyon tamamlandı.")

# Model girdisi olarak sadece 'input_ids', 'attention_mask' ve 'token_type_ids' (BERT için) gerekir.
# Orijinal 'text' sütunu artık gereksizdir ve bellekten tasarruf etmek için kaldırılır.
print("Gereksiz sütunlar kaldırılıyor...")
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["text"])
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["text"]) # Doğrulama setinden de kaldırılır
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["text"]) # Test setinden de kaldırılır
print("Sütunlar kaldırıldı.")


print("\nTokenize edilmiş eğitim seti:\n", tokenized_train_dataset)
print("Tokenize edilmiş doğrulama seti:\n", tokenized_val_dataset) # Doğrulama setinin bilgisini de göster
print("Tokenize edilmiş test seti:\n", tokenized_test_dataset)

# İlk token'lanmış eğitim örneğini incele (isteğe bağlı)
# Bu, tokenizasyonun düzgün çalıştığını ve modelin beklediği formatta (input_ids, attention_mask, labels) olduğunu doğrulamak için faydalıdır.
print("\nİlk token'lanmış eğitim örneği:")
print(tokenized_train_dataset[0])


Veri setleri token'lara ayrılıyor...


Map: 100%|████████████████████████████████████████████████████████████████████| 70/70 [00:00<00:00, 2467.07 examples/s]

Tokenizasyon tamamlandı.
Gereksiz sütunlar kaldırılıyor...
Sütunlar kaldırıldı.

Tokenize edilmiş eğitim seti:
 Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 560
})
Tokenize edilmiş doğrulama seti:
 Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 63
})
Tokenize edilmiş test seti:
 Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 70
})

İlk token'lanmış eğitim örneği:
{'labels': 0, 'input_ids': [2, 2048, 17153, 13349, 1992, 7539, 19709, 1990, 2012, 2605, 4857, 1973, 1996, 2339, 2045, 1023, 3288, 2839, 1024, 3956, 2330, 2570, 11866, 6440, 2975, 5512, 6301, 1973, 6922, 2165, 3765, 1990, 23963, 10910, 3176, 6101, 11920, 3926, 23721, 1025, 17153, 1048, 3445, 70, 1027, 6717, 2605, 4857, 2025, 3819, 28196, 6453, 6126, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [24]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np # Sayısal işlemler için NumPy kütüphanesi
from sklearn.metrics import precision_recall_fscore_support, accuracy_score # Model performans metriklerini hesaplamak için

# 1. Adım: Performans Metriklerini Tanımla (compute_metrics fonksiyonu)
# Trainer tarafından her değerlendirme adımında çağrılacak olan metrik hesaplama fonksiyonu.
# 'p' parametresi, modelin tahminlerini (predictions) ve gerçek etiketleri (label_ids) içerir.
def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids # Tahminleri ve gerçek etiketleri al
    preds = np.argmax(predictions, axis=1) # En yüksek olasılığa sahip sınıfı tahmin olarak seç (logitlerden sınıf indeksine dönüştür)

    # İkili sınıflandırma için (0: doğru, 1: yanlış) kesinlik, geri çağırma ve F1 skorunu hesapla.
    # 'average='binary'' ikili sınıflandırma için metriklerin hesaplandığını belirtir.
    # 'pos_label=1' pozitif sınıfın (yani 'yanlış' haberin) 1 olduğunu belirtir.
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary', pos_label=1)
    acc = accuracy_score(labels, preds) # Doğruluk (Accuracy) oranını hesapla
    
    # Hesaplanan metrikleri bir sözlük olarak döndür. Trainer bu formatı bekler.
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}

# 2. Adım: Eğitim Argümanlarını Belirle
# TrainingArguments sınıfı, eğitim sürecinin nasıl yapılandırılacağını belirler.
training_args = TrainingArguments(
    output_dir="./results", # Model çıktıları ve checkpoint'lerin kaydedileceği dizin
    eval_strategy="epoch", # Değerlendirmeyi her epoch sonunda yap
    save_strategy="epoch", # Her epoch sonunda model checkpoint'ini kaydet
    learning_rate=1e-5, # Modelin öğrenme oranı (BERT ince ayarı için genellikle küçük bir değer kullanılır)
    per_device_train_batch_size=8, # Her GPU/CPU cihazı için eğitim batch boyutu
    per_device_eval_batch_size=8, # Her GPU/CPU cihazı için değerlendirme batch boyutu
    num_train_epochs=3, # Toplam eğitim epoch sayısı
    weight_decay=0.01, # Ağırlık bozunumu (L2 regülarizasyonu), aşırı öğrenmeyi engellemeye yardımcı olur
    logging_dir='./logs', # Logların kaydedileceği dizin
    logging_steps=100, # Loglama bilgilerini (kayıp vb.) kaç adımda bir yazdırılacağını belirler
    # save_total_limit=2, # Yorum satırı: Kaydedilecek en fazla checkpoint sayısını belirler (örn: en iyi 2 model)
    load_best_model_at_end=True, # Eğitimin sonunda doğrulama setinde en iyi performansı gösteren modeli yükle
    metric_for_best_model="f1", # En iyi modeli belirlemek için kullanılacak metrik
    greater_is_better=True, # Seçilen metrik (f1) için daha yüksek değerin daha iyi olduğunu belirtir
)

# 3. Adım: Trainer Objesini Tanımla
# Trainer sınıfı, PyTorch modellerini eğitmek ve değerlendirmek için yüksek seviyeli bir API sağlar.
trainer = Trainer(
    model=model, # Eğitilecek model
    args=training_args, # Eğitim argümanları
    train_dataset=tokenized_train_dataset, # Eğitim veri seti (daha önce token'lara ayrılmış)
    eval_dataset=tokenized_val_dataset, # Doğrulama veri seti (eğitim sırasında performansı izlemek için)
    compute_metrics=compute_metrics, # Metrik hesaplama fonksiyonu
)

# 4. Adım: Modeli Eğit!
print("\nModel eğitimi başlatılıyor...")
trainer.train() # Eğitim sürecini başlatır
print("Model eğitimi tamamlandı.")

# 5. Adım: Modelin Son Değerlendirmesini Yap
print("\nTest seti üzerinde model değerlendiriliyor...")
# Eğitilmiş modeli daha önce ayrılmış test seti üzerinde değerlendirir.
# Bu, modelin yeni, daha önce görmediği veriler üzerindeki genelleme yeteneğini ölçer.
evaluation_results = trainer.evaluate(eval_dataset=tokenized_test_dataset)
print("Değerlendirme Sonuçları:", evaluation_results)


Model eğitimi başlatılıyor...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.313441,0.904762,0.878788,0.935484,0.906250
2,0.277300,0.301536,0.920635,0.933333,0.903226,0.918033
3,0.228700,0.337417,0.920635,0.964286,0.870968,0.915254


C:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
C:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Model eğitimi tamamlandı.

Test seti üzerinde model değerlendiriliyor...


C:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Değerlendirme Sonuçları: {'eval_loss': 0.44081926345825195, 'eval_accuracy': 0.8714285714285714, 'eval_precision': 0.8378378378378378, 'eval_recall': 0.9117647058823529, 'eval_f1': 0.8732394366197183, 'eval_runtime': 1.4274, 'eval_samples_per_second': 49.04, 'eval_steps_per_second': 6.305, 'epoch': 3.0}


In [26]:
import torch # Derin öğrenme operasyonları için PyTorch kütüphanesi

# 1. Tahmin Fonksiyonu Yaz
# Verilen bir metnin yanlış olma olasılığını tahmin eden fonksiyon.
# 'clean_text' fonksiyonu ve 'tokenizer' ile 'model' objeleri daha önceki hücrelerde tanımlandı.
def predict_fake_news(text, model, tokenizer, max_length=256):
    # Tahmin yapılacak metni, eğitim verisiyle aynı şekilde temizle.
    # 'clean_text' fonksiyonunun bu hücreye erişilebilir olması gerekmektedir.
    cleaned_text = clean_text(text) 

    # Temizlenmiş metni tokenize et.
    # `return_tensors="pt"`: PyTorch tensorları olarak çıktı döndür.
    # `truncation=True`: Metin max_length'ten uzunsa kes.
    # `padding="max_length"`: Metin max_length'ten kısaysa doldur.
    # `max_length=max_length`: Tokenize edilmiş metnin maksimum uzunluğu.
    inputs = tokenizer(cleaned_text, return_tensors="pt", truncation=True, padding="max_length", max_length=max_length)

    # DİKKAT: Buradaki girinti hatası düzeltildi!
    if torch.cuda.is_available():
        # Girdileri (input_ids, attention_mask) GPU'ya taşı
        inputs = {k: v.to('cuda') for k, v in inputs.items()}
        # Modeli GPU'ya taşı (eğer henüz taşınmadıysa veya CPU'ya geri döndüyse)
        # Trainer genellikle modeli GPU'da bırakır, ancak bağımsız bir tahmin fonksiyonunda bu kontrolü yapmak iyidir.
        if model.device.type != 'cuda': 
            model.to('cuda')
            
    # Model ile tahmin yap.
    # `torch.no_grad()`: Tahmin yaparken gradyan hesaplamalarını devre dışı bırakır,
    # bu da bellek kullanımını azaltır ve çıkarım hızını artırır.
    with torch.no_grad(): 
        outputs = model(**inputs)

    # Modelin ham çıktıları (logitler) alınır ve olasılıklara dönüştürülür.
    # `torch.softmax(logits, dim=1)`: Logitleri olasılık dağılımına çevirir.
    # `dim=1`: Sınıf boyutunda softmax uygula (yani her örnek için sınıf olasılıkları).
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)

    # Yanlış olma olasılığı, 'fake' etiketi için atadığımız indeks olan 1'e karşılık gelir.
    # `[:, 1]` ikinci sınıfın (yani 'yanlış' sınıfının) olasılığını seçer.
    # `.item()`: Tensor içindeki tek bir değeri Python sayısına dönüştürür.
    fake_prob = probabilities[:, 1].item()
    
    return fake_prob

# 2. Örnek Metinler Üzerinde Tahminleri Test Et
print("\n--- Örnek Metinler Üzerinde Tahminler ---")

# Modelin performansını göstermek için kullanılacak örnek metinler listesi.
test_texts = [
    "Bilim insanları Mars'ta devasa bir uzaylı üssü keşfetti.", # Yanlış olması beklenen (kurgu)
    "2024 olimpiyatlarına Paris ev sahipliği yapacak.", # Doğru olması beklenen (gerçek bilgi)
    "Dünya'nın çekirdeğindeki manyetik alan, bu hafta itibarıyla yön değiştirerek tüm pusulaları bozdu.", #Yanlış
    "Birleşmiş Milletler Güvenlik Konseyi, Suriye'ye yönelik insani yardımın süresini altı ay daha uzatma kararı aldı.", #Doğru
    "Güneş Sistemi'nde yeni bir gezegen bulundu ve adı 'Patoz' konuldu.", # Yanlış olması beklenen (kurgu)
    "Avrupa Merkez Bankası, artan enflasyonla mücadele kapsamında faiz oranlarında 25 baz puanlık artışa gitti.", #Doğru
    "Yeni yapılan bir araştırmaya göre, gece 4 saat uyuyanlar, 8 saat uyuyanlara göre daha uzun yaşıyor.",#Yanlış
    "Diplomatlar, Afrika'daki kuraklık kriziyle mücadele için uluslararası bir fon oluşturulması konusunda anlaşmaya vardı." ,#Doğru
    "Hükümet, 2025'ten itibaren tüm araçlarda suyla çalışan motorların kullanılmasını zorunlu hale getirecek." ,#Yanlış
    "Uzay araştırmacıları, Ay'ın üzerinde 1500 yıl önce inşa edilmiş piramit benzeri yapılar keşfetti."#Yanlış
]

# Sayısal etiket ID'lerini okunabilir metin etiketlerine dönüştürmek için sözlük.
id_to_label = {0: 'Doğru', 1: 'Yanlış'}

# Her bir örnek metin üzerinde tahmin yap ve sonuçları göster.
for text in test_texts:
    # predict_fake_news fonksiyonunu çağırarak yanlış olma olasılığını al.
    risk_score = predict_fake_news(text, model, tokenizer)
    # Risk puanı 0.5'ten büyükse 'Yanlış' (1), değilse 'Doğru' (0) olarak sınıflandır.
    predicted_label_id = 1 if risk_score > 0.5 else 0 
    predicted_label = id_to_label[predicted_label_id] # Sayısal etiketi metin etikete dönüştür.

    print(f"\nMetin: '{text}'")
    print(f"Yanlış Olma Olasılığı (Risk Puanı): {risk_score:.4f}") # Olasılığı 4 ondalık basamakla yazdır.
    print(f"Tahmini Etiket: {predicted_label}")

    # Öneri Mekanizması: Risk puanına göre kullanıcıya farklı uyarılarda bulun.
    if risk_score > 0.7: # Yüksek riskli (örn. %70 üzeri)
        print(">> Bu haberin yanlış olma olasılığı YÜKSEK. Lütfen güvenilir kaynaklardan teyit edin (örn. teyit.org).")
    elif risk_score > 0.4: # Orta riskli (örn. %40-70 arası)
        print(">> Bu haberin doğruluğunu teyit etmeniz ÖNERİLİR. Kaynağını kontrol edin.")
    else: # Düşük riskli (örn. %40 altı)
        print(">> Bu haberin doğru olma olasılığı YÜKSEK görünüyor. Yine de her zaman kritik düşünün.")

# Siber Okuryazarlık İpuçları (Metinsel)
# Kullanıcıların dezenformasyonu tanıma ve eleştirel düşünme becerilerini geliştirmelerine yardımcı olacak pratik ipuçları.
print("\n--- Siber Okuryazarlık İpuçları ---")
print("1. Haber başlıkları çok iddialı veya duygusal ise şüphelenin.")
print("2. Haberin kaynağını (yazar, yayıncı site) kontrol edin. Güvenilir mi?")
print("3. Haberin tarihi önemli mi? Eski bir haber güncel gibi mi sunuluyor?")
print("4. Farklı, bilinen haber kaynaklarından aynı bilgiyi teyit etmeye çalışın.")
print("5. Yazım ve dilbilgisi hatalarına dikkat edin. Profesyonel yayınlarda bu tür hatalar nadirdir.")
print("6. Görselleri tersine görsel arama ile kontrol edin. Farklı bir bağlamda kullanılmış olabilirler.")


--- Örnek Metinler Üzerinde Tahminler ---

Metin: 'Bilim insanları Mars'ta devasa bir uzaylı üssü keşfetti.'
Yanlış Olma Olasılığı (Risk Puanı): 0.9048
Tahmini Etiket: Yanlış
>> Bu haberin yanlış olma olasılığı YÜKSEK. Lütfen güvenilir kaynaklardan teyit edin (örn. teyit.org).

Metin: '2024 olimpiyatlarına Paris ev sahipliği yapacak.'
Yanlış Olma Olasılığı (Risk Puanı): 0.2556
Tahmini Etiket: Doğru
>> Bu haberin doğru olma olasılığı YÜKSEK görünüyor. Yine de her zaman kritik düşünün.

Metin: 'Dünya'nın çekirdeğindeki manyetik alan, bu hafta itibarıyla yön değiştirerek tüm pusulaları bozdu.'
Yanlış Olma Olasılığı (Risk Puanı): 0.8493
Tahmini Etiket: Yanlış
>> Bu haberin yanlış olma olasılığı YÜKSEK. Lütfen güvenilir kaynaklardan teyit edin (örn. teyit.org).

Metin: 'Birleşmiş Milletler Güvenlik Konseyi, Suriye'ye yönelik insani yardımın süresini altı ay daha uzatma kararı aldı.'
Yanlış Olma Olasılığı (Risk Puanı): 0.0142
Tahmini Etiket: Doğru
>> Bu haberin doğru olma olasılığı YÜKSEK g